In [1]:
import sys
sys.path.append('../src')

import network as nw
import torch
import generate_kvadraturen as gk
import bus
import road as rd
import traffic_lights as tl
import junction as jn
import numpy as np
import loading_json as load
import json

First we read in the configuration of the network from a json file before solving the conservation law

In [2]:
f = open("../kvadraturen_networks/1_1.json")
data = json.load(f)
f.close()
T = data["T"]
# T = 30
N = data["N"]
speed_limits = data["speed_limits"] # Nested list
control_points = data["control_points"] # Nested list
cycle_times = data["cycle_times"] # Nested list

In [3]:
bus_network = gk.generate_kvadraturen_roundabout_w_params(T, N, speed_limits, control_points, cycle_times,
                                                          track_grad=False)

In [4]:
densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

Bus 2 reached bus stop 0 at time 26.200014114379883, should wait for 29.826133728027344 seconds
Bus 2 reached bus stop 1 at time 117.89981079101562, should wait for 29.562639236450195 seconds
Bus 2 reached bus stop 2 at time 172.24998474121094, should wait for 29.89476203918457 seconds
Bus 1 reached bus stop 0 at time 203.5999755859375, should wait for 29.689308166503906 seconds
Bus 3 reached bus stop 0 at time 270.9000244140625, should wait for 29.991413116455078 seconds
Bus 1 reached bus stop 1 at time 274.95013427734375, should wait for 29.638992309570312 seconds
Bus 3 reached bus stop 1 at time 355.45001220703125, should wait for 29.65973472595215 seconds
Bus 2 reached bus stop 3 at time 369.33197021484375, should wait for 29.800180435180664 seconds
Bus 1 reached bus stop 2 at time 375.33819580078125, should wait for 29.84400749206543 seconds
Bus 3 reached bus stop 2 at time 409.8031311035156, should wait for 29.850933074951172 seconds
Bus 1 reached bus stop 3 at time 441.350036621

Then we convert the format of the result from the simulation to be able to write it to a json file

In [5]:
densities_loaded = load.convert_from_tensor(densities)
queues_loaded = load.convert_from_tensor(queues)
bus_lengths_loaded = load.convert_from_tensor(bus_lengths)
bus_delay_dict = {i : [float(l.detach()) for l in bus_delays[i]] for i in range(len(bus_delays))}

Writing the results to a json file

In [6]:
import json
with open("kvadraturen_500_orig.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, queues_loaded, bus_lengths_loaded, bus_delay_dict]))

Store also the results only for interior nodes

In [7]:
f = open("kvadraturen_500_orig.json")
data = json.load(f)
f.close()
densities = data[0]
queues = data[1]
bus_lengths = data[2]
bus_delays = data[3]

In [8]:
shortened_densities = {}
for road_id in densities.keys():
    shortened_densities[road_id] = {}
    for t in densities[road_id].keys():
        old_dens = densities[road_id][t]
        new_dens = [old_dens[i] for i in range(2, len(old_dens)-2)]
        shortened_densities[road_id][t] = new_dens      

In [9]:
with open("kvadraturen_500_temp_opt_internal.json", 'w') as fd:
    fd.write(json.dumps([shortened_densities, queues, bus_lengths, bus_delays]))